In [0]:
import pandas as pd
import numpy as np
from os.path import join

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [0]:
# data from https://www.energidataservice.dk/tso-electricity/elspotprices 
import requests

new_limit = 100000
url = f'https://api.energidataservice.dk/dataset/Elspotprices?limit={new_limit}'
response = requests.get(url)

if response.status_code == 200:
    result = response.json()
    records = result.get('records', [])

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

In [0]:
len(records)

100000

In [0]:
import json
with open('./data.json', 'w') as json_file:
    json.dump(records, json_file)

In [0]:
df = pd.read_json('./data.json')
df.head()

,HourUTC,HourDK,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2023-12-14T22:00:00,2023-12-14T23:00:00,DK1,638.020,85.570
1,2023-12-14T22:00:00,2023-12-14T23:00:00,DK2,638.020,85.570
2,2023-12-14T22:00:00,2023-12-14T23:00:00,NO2,619.530,83.090
3,2023-12-14T22:00:00,2023-12-14T23:00:00,SE3,638.020,85.570
4,2023-12-14T22:00:00,2023-12-14T23:00:00,SE4,638.020,85.570


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   HourUTC       100000 non-null  object 
 1   HourDK        100000 non-null  object 
 2   PriceArea     100000 non-null  object 
 3   SpotPriceDKK  100000 non-null  float64
 4   SpotPriceEUR  100000 non-null  float64
dtypes: float64(2), object(3)
memory usage: 3.8+ MB


In [0]:
# Use only UTC hour and EUR Price columns
df.columns = [i.lower() for i in df.columns.tolist()]
df = df[['hourutc', 'pricearea', 'spotpriceeur']]
df.columns = ['date', 'area', 'spot_price']

date_format = "%Y-%m-%dT%H:%M:%S"
df['date'] = pd.to_datetime(df['date'], format=date_format)
df.head()

,date,area,spot_price
0,2023-12-14 22:00:00,DK1,85.570
1,2023-12-14 22:00:00,DK2,85.570
2,2023-12-14 22:00:00,NO2,83.090
3,2023-12-14 22:00:00,SE3,85.570
4,2023-12-14 22:00:00,SE4,85.570


In [0]:
df['area'].unique()

array(['DK1', 'DK2', 'NO2', 'SE3', 'SE4', 'SYSTEM', 'DE'], dtype=object)

In [0]:
import plotly.express as px
## hourly full time graph
px.line(df, x=df['date'], y=df['spot_price'], color='area')

In [0]:
## checking specific period 
sub = df[
        (df['date']>='2023-12-03') &
        (df['date']<='2023-12-09')
        ]           
## hourly full time graph
px.line(sub, x=sub['date'], y=sub['spot_price'], color='area')

### Simple time series model with prophet 

In [0]:
import warnings
warnings.filterwarnings("ignore")
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import add_changepoints_to_plot

In [0]:
## only use DK1, NO2, SE3, DE
raw_df = df[df['area'].isin(['DK1', 'NO2', 'SE3', 'DE'])]
raw_df['area'] = raw_df['area'].apply(lambda x: x[:2])
raw_df['area'].unique()

array(['DK', 'NO', 'SE', 'DE'], dtype=object)

In [0]:

def calculate_metrics(actual, predicted):
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    r_squared = r2_score(actual, predicted)
    return mape, r_squared

results = []

for country in raw_df['area'].unique():
    df_area = raw_df[raw_df['area'] == country].copy()
    df_area = df_area.sort_values('date').reset_index(drop=True)

    # Prepare DataFrame with 'ds' and 'y' columns for Prophet
    df_area_prophet = df_area.rename(columns={'date': 'ds', 'spot_price': 'y'})

    # Train and forecast using Prophet
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, seasonality_mode='additive')
    model.add_country_holidays(country_name=country)  
    model.fit(df_area_prophet)

    # Forecast 12 hours into the future
    future = model.make_future_dataframe(periods=48, freq='H')  # following 12 hours prediction
    forecast = model.predict(future)

    result = pd.DataFrame({
            'date': forecast['ds'],
            'area': country,
            'actual_spot_price': df_area['spot_price'],
            'prediction_spot_price': forecast['yhat'],
            'baseline_pred': forecast['yhat_lower'],
            'seasonality_pred': forecast['yhat_upper']
         })
    # if some spot prices has nan
    result['actual_spot_price'].fillna(method='ffill', inplace=True)
    results.append(result)

20:00:39 - cmdstanpy - INFO - Chain [1] start processing
20:00:48 - cmdstanpy - INFO - Chain [1] done processing
20:00:55 - cmdstanpy - INFO - Chain [1] start processing
20:01:05 - cmdstanpy - INFO - Chain [1] done processing
20:01:11 - cmdstanpy - INFO - Chain [1] start processing
20:01:16 - cmdstanpy - INFO - Chain [1] done processing
20:01:22 - cmdstanpy - INFO - Chain [1] start processing
20:01:31 - cmdstanpy - INFO - Chain [1] done processing


In [0]:
final_res = pd.concat(results)
final_res.head()

,date,area,actual_spot_price,prediction_spot_price,baseline_pred,seasonality_pred
0,2022-04-28 10:00:00,DK,208.000,221.164,137.946,311.046
1,2022-04-28 11:00:00,DK,200.090,221.169,128.381,307.028
2,2022-04-28 12:00:00,DK,197.050,221.153,120.526,311.677
3,2022-04-28 13:00:00,DK,196.300,221.115,130.821,316.631
4,2022-04-28 14:00:00,DK,197.130,221.056,139.443,309.280


### Calculate r_squared of each area 

In [0]:
from sklearn.metrics import r2_score

In [0]:
for country in final_res['area'].unique():
    sub = final_res[final_res['area']==country]
    r_squared = r2_score(sub['actual_spot_price'], sub['prediction_spot_price'])
    print(f'R-squared for AREA: {country}: {r_squared}')

R-squared for AREA: DK: 0.7237777653394262
R-squared for AREA: NO: 0.8362059452208863
R-squared for AREA: SE: 0.5215092989234869
R-squared for AREA: DE: 0.7428070878131825


In [0]:
# Check best area = NO
df_no = final_res[final_res['area']=='NO']
df_no.head(1)

,date,area,actual_spot_price,prediction_spot_price,baseline_pred,seasonality_pred
0,2022-04-28 10:00:00,NO,201.550,217.066,152.855,280.881


In [0]:
# Compare prediction with real
sub_no = df_no[
        (df_no['date']>='2023-12-03') &
        (df_no['date']<='2023-12-10')
        ]  
fig = px.line(sub_no, x=sub_no['date'], y=['actual_spot_price','prediction_spot_price', 'baseline_pred', 'seasonality_pred'])
fig.show()